In [0]:
from pyspark.sql.functions import col
from pyspark.sql import functions as F
from pyspark.sql.functions import *
from datetime import datetime, timedelta
import pandas as pd
ssesh = spark

In [0]:
df1=spark.sql("""
              select
iif.report as IIF_report,
iif.*,
to_date (iif.report, 'yyyyMMdd') AS IIF_report_date,
to_date (iif.APLCAPRO, 'yyyyMMdd') AS iif_approved_date,
SUBSTRING(iif.APLCAPRO,1,4) as iif_approved_year,
SUBSTRING(iif.APLCAPRO,5,2) as iif_approved_month,
SUBSTRING(las.approved_date_surrkey,1,4) as las_approved_year,

/*testing dates data*/
emili.APLCAPRO as emili_approval_date,
carl.APLCAPRO as carl_approvaldate,
lad.application_approved_system_dt as lad_application_approved_system_dt,
las.approved_date_surrkey as las_approved_date_surrkey,
las.ever_approved_date_surrkey as las_ever_approved_date_surrkey,
preups.APLCAPRO preups_approv_date, 

to_date ((
    CASE
	WHEN iif.pdtcat = 3 THEN COALESCE(iif.APLCAPRO, carl.APLCAPRO, crm.cmhc_firstapprovaldate, preups.APLCAPRO, to_date ( las.approved_date_surrkey, 'yyyyMMdd'))
    WHEN iif.pdtcat = 2 THEN COALESCE(iif.APLCAPRO, port.APLCAPRO, emili.APLCAPRO, preups.APLCAPRO, to_date ( las.approved_date_surrkey, 'yyyyMMdd'))
    WHEN iif.pdtcat = 1 THEN COALESCE(iif.APLCAPRO, emili.APLCAPRO, preups.APLCAPRO,to_date ( las.approved_date_surrkey, 'yyyyMMdd'))
END),'yyyyMMdd') AS approved_date_comb, --creation of the approval date 
left(approved_date_comb,4) as approved_year,
SUBSTRING(approved_date_comb,6,2) as approved_month,

case
	when lad.low_ratio_mortgage_tcd in ('**', '03') then "High Ratio" else "Low Ratio" end as low_ratio_ind,

lad.low_ratio_insurable_type_desc_en, 
lad.low_ratio_ever_ind,

case 
when SUBSTRING(report,5,4) = '1231' then SUBSTRING(iif.report,1,4) end as year_end_reports,

case when iif.pdtcat = 1 then "HO"
  when iif.pdtcat = 2 then "Port"
  else "Multi" end as Product,
  
/*drop off rate help*/
CASE
WHEN ROW_NUMBER() OVER (PARTITION BY iif.LNACCTNO ORDER BY iif.report DESC) = 1 THEN 1 ELSE 0 END AS IsLatestReportedDate,

/*MU Select, Flex and Market products*/
crm.cmhc_mliflexflag as MLI_Select_flag,
CASE when crm.cmhc_isaffordablerentalhousing="True" and crm.cmhc_receiveddate < "2022-03-07 00:00:00.0" then "True"
     when crm.cmhc_isaffordablerentalhousing="False" and crm.cmhc_receiveddate < "2022-03-07 00:00:00.0" then "False"
  ELSE NULL
end as MLI_Flex_flag, 
CASE
  WHEN (MLI_Flex_flag IS NOT TRUE AND MLI_Select_flag is not TRUE) then "True"
     ELSE "False"
END AS MLI_Market_Product_flag,

/*getting MU postal codes*/
replace(SUBSTRING_INDEX(crm.cmhc_locationidname, ",", -1),' ', '') as Property_Postal_Code_raw,
replace(SUBSTRING_INDEX(loc.cmhc_postalcode, ",", -1),' ', '') as cmhc_postalcode2,/*MU poastals from crosslob*/
trim(Substring_index(Substring_index(crm.cmhc_locationidname, ',', 2), ',', -1)) AS cmhc_propertyaddress_w2,
trim(Substring_index(Substring_index(trim(crm.cmhc_locationidname), ',', 3), ',', -1)) AS cmhc_propertyaddress_w3,

CASE when cmhc_propertyaddress_w2 = cmhc_propertyaddress_w3 then RIGHT(Property_Postal_Code_raw,6)
else Property_Postal_Code_raw
END AS Property_Postal_Code,

CASE
    WHEN iif.pdtcat = 3 THEN REPLACE(COALESCE(pr.property_address_postal_cde, cmhc_postalcode2, Property_Postal_Code, preups.PRE_PAPOSTCD, iif.postcode, carl.papostcd), ' ', '')
    WHEN iif.pdtcat = 2 THEN REPLACE(COALESCE(port.vpapstcd, port.papostcd, pr.property_address_postal_cde, preups.PRE_PAPOSTCD, iif.postcode), ' ', '')
    WHEN iif.pdtcat = 1 THEN REPLACE(COALESCE(pr.property_address_postal_cde, preups.PRE_PAPOSTCD, iif.postcode), ' ', '')
END AS postal_cde_comb, --creation of the combined postal code

/*METMAJOR code*/
case 
	when preups = 1 then substr(preups.PRE_METMJRCD,1,4)
	/*Portfolio*/
	when iif.pdtcat = 2 and port.mtmjrcde <> '' then port.mtmjrcde 
	/*Homeowner and multi*/
	when pr.current_metropolitan_major_area_cde not in ('','**') then pr.current_metropolitan_major_area_cde
	else pr.reported_metropolitan_major_area_cde /*New in rebuild*/
end as met_major_cde_comb,

case when cast(trim(met_major_cde_comb) as CHAR(4)) = cast(trim(pr.current_metropolitan_major_area_cde) as CHAR(4)) then pr.current_metropolitan_major_area_nm
	else 'No match' end as CMA_Met_Maj,
 
CASE
    /* Multi */
    WHEN iif.pdtcat = 3 AND postal_cde_comb = '' OR postal_cde_comb is null THEN 'Not Available'
    WHEN iif.pdtcat = 3 AND SUBSTRING(postal_cde_comb, 2, 1) in ('0',0) THEN 'Rural'
   	WHEN iif.pdtcat = 3 AND SUBSTRING(postal_cde_comb, 2, 1) <> '0' THEN 'Urban'
 ELSE 'Urban'
end as MU_urban_rural_desc,

/*vprovcd used codes 00-13 until 2020-09, then in 2020-10 switched to codes 10-62.
Convert them all to 00-13 for consistency.*/
case
	when port.vprovcd = '10' then '00'
	when port.vprovcd = '11' then '01'
	when port.vprovcd = '12' then '02'
	when port.vprovcd = '13' then '03'
	when port.vprovcd = '24' then '04'
	when port.vprovcd = '35' then '05'
	when port.vprovcd = '46' then '06'
	when port.vprovcd = '47' then '07'
	when port.vprovcd = '48' then '08'
	when port.vprovcd = '59' then '09'
	when port.vprovcd = '60' then '11'
	when port.vprovcd = '61' then '12'
	when port.vprovcd = '62' then '13'
	else port.vprovcd
end as upsmort_vprovcd_mod,

/*
Testing showed that historically, the reported and current versions of the province variables
matched in 99.9% of homeowner loans. Since 2014, they match at 100%.
For portfolio loans, upsmort matches reported_stats_canada_province_c.
For multi-unit loans, ERIC usually matches with reported, but ERIC tends to be populated even when reported = **.
SPSS gave priority to reported version followed by current. 
*/

/*for MU using CRM data*/
CASE WHEN crm.cmhc_provinceterritory_en IN ('Newfoundland and Labrador') then '10'
WHEN crm.cmhc_provinceterritory_en IN ('Prince Edward Island') then '11'
WHEN crm.cmhc_provinceterritory_en IN ('Nouvelle Ecosse','Nova Scotia') then '12'
WHEN crm.cmhc_provinceterritory_en IN ('New Brunswick','Nouveau Brunswick') then '13'
WHEN crm.cmhc_provinceterritory_en IN ('Québec', 'Quebec') then '24'
WHEN crm.cmhc_provinceterritory_en IN ('Ontario') then '35'
WHEN crm.cmhc_provinceterritory_en IN ('Manitoba') then '46'
WHEN crm.cmhc_provinceterritory_en IN ('Saskatchewan') then '47'
WHEN crm.cmhc_provinceterritory_en IN ('Alberta') then '48'
WHEN crm.cmhc_provinceterritory_en IN ('British Columbia','British Colombia') then '59'
WHEN crm.cmhc_provinceterritory_en IN ('Yukon') then  '60'
WHEN crm.cmhc_provinceterritory_en IN ('Northwest Territories','N.W.T.','NWT') then '61'
WHEN crm.cmhc_provinceterritory_en IN ('Nunavut') then '62'
ELSE NULL
END AS MU_Property_scprovcd,

case
	when trim(preups.PRE_SCPROVCD2)<>'' then substr(preups.PRE_SCPROVCD2,1,2)
	when trim(MU_Property_scprovcd) <>'' then MU_Property_scprovcd
	when trim(pr.reported_stats_canada_province_cde) not in ('','**') then pr.reported_stats_canada_province_cde
	when trim(pr.current_stats_canada_province_cde) not in ('','**') then pr.current_stats_canada_province_cde
	when upsmort_vprovcd_mod<>'' then upsmort_vprovcd_mod
	else '99'
end as prov_cde_comb,

/*Convert the combined province code to the Stats Can version: the first digit represents the region*/
case
	/*Atlantic provinces*/
	when prov_cde_comb in ('00','01','02','03',0,1,2,3) then cast(prov_cde_comb as int) + 10
	/*Quebec*/
	when prov_cde_comb in ('04',4) then 24
	/*Ontario*/
	when prov_cde_comb in ('05',5) then 35
	/*Prairie provinces*/
	when prov_cde_comb in ('06','07','08',6,7,8) then cast(prov_cde_comb as int) + 40
	/*BC*/
	when prov_cde_comb in ('09',9) then 59
	/*Northern territories*/
	when prov_cde_comb = '11' then 60
	when prov_cde_comb = '12' then 61
	when prov_cde_comb = '13' then 62
	when prov_cde_comb in ('10','11','12','13','24','35','46','47','48','59','60','61','62') then prov_cde_comb
	/*Unknown*/
	else 99
end as sc_prov_cde_comb,
/*Create the province group variable for reporting (northern territories grouped together)*/

case
	when sc_prov_cde_comb in (60,61,62) then 60
	else sc_prov_cde_comb
end as prov_cde_grp,

case
	when  prov_cde_grp = 10 then 'Newfoundland and Labrador'
	when  prov_cde_grp = 11 then 'Prince Edward Island'
	when  prov_cde_grp = 12 then 'Nova Scotia'
	when  prov_cde_grp = 13 then 'New Brunswick'
	when  prov_cde_grp = 24 then 'Quebec'
	when  prov_cde_grp = 35 then 'Ontario'
	when  prov_cde_grp = 46 then 'Manitoba'
	when  prov_cde_grp = 47 then 'Saskatchewan'
	when  prov_cde_grp = 48 then 'Alberta'
	when  prov_cde_grp = 59 then 'British Columbia'
	when  prov_cde_grp = 60 then 'Yukon/NWT/Nunavut'   
	else 'Not Available'
end as prov_name,

/*Region*/
case 
	when  prov_cde_comb in ('00','01','02','03') then 'Atlantic'
	when  prov_cde_comb = '04' then 'Quebec'
	when  prov_cde_comb = '05' then 'Ontario'
	when  prov_cde_comb in ('06','07','08','11','12','13') then 'Prairies & North'
	when  prov_cde_comb = '09' then 'British Columbia'
	else 'Not Available'
end as region,

  /* CASE
  Multi
    WHEN iif.pdtcat = 3 (AND postal_cde_comb = '' OR postal_cde_comb is null) THEN 'Not Available'
    WHEN iif.pdtcat = 3 AND SUBSTRING(postal_cde_comb, 2, 1) in ('0',0) THEN 'Rural'
   	WHEN iif.pdtcat = 3 AND SUBSTRING(postal_cde_comb, 2, 1) <> '0' THEN 'Urban'
    /*  Portfolio & Homeowner */
    WHEN met_major_cde_comb IN ('', '**') OR TRIM(pr.urban_rural_cde) = '' THEN 'Not Available'
    /* Population < 10K and remote or rural */
    WHEN met_major_cde_comb = '9000' AND TRIM(pr.urban_rural_cde) IN ('2', '91') THEN 'Rural'
    ELSE 'Urban'
end as urban_rural_desc,
 */

CASE
    WHEN iif.pdtcat = 3 then MU_urban_rural_desc
    when iif.pdtcat in (2,1) and (met_major_cde_comb IN ('', '**') OR TRIM(pr.urban_rural_cde) = '') THEN 'Not Available'
    WHEN met_major_cde_comb = '9000' AND TRIM(pr.urban_rural_cde) IN ('2', '91') THEN 'Rural'
    ELSE 'Urban'
END AS urban_rural_desc,

/*Dwelling type. Removed CARL.*/
CASE
  WHEN preups = 1 THEN CONCAT('00', CAST(preups.pre_dweltype AS STRING))
  WHEN TRIM(proj.cmhc_dwellingtype_en) NOT IN ('', '**') THEN TRIM(proj.cmhc_dwellingtype_en)
  WHEN TRIM(pr.dwelling_tcd) NOT IN ('', '**') THEN pr.dwelling_tcd
  ELSE ''
END AS dwelling_tcd_comb,

case
	when dwelling_tcd_comb in ('', '**') then 'Not Available'
	when dwelling_tcd_comb = '001' then 'Single Detached'
	when dwelling_tcd_comb = '002' then 'Semi-Detached'
	when dwelling_tcd_comb in ('003','008','018') then 'Duplex / Triplex / Quadruplex'
	when dwelling_tcd_comb = '004' then 'Row House'
	when dwelling_tcd_comb = '005' then 'Apartment'
	else 'Other'																	

end as dwelling_grp,


pr.urban_rural_cde as urban_rural_cde_prop_ac_dim ,
pr.urban_rural_desc_en as urban_rural_desc_en_prop_ac_dim,

case
  when substr(trim(intype),1,1) in ('1','F','f') then '1 - Fixed'
  when substr(trim(intype),1,1) in ('2','V','v') then '2 - Variable'
  when substr(trim(intype),1,1) = '3' then '3 - Adjustable'
  when substr(trim(intype),1,1) = '4' then '4 - Capped Variable'
  else 'Unknown'
end as interest_type,

case
    when cast(payfreq as int) in (1, 12) or upper(trim(payfreq)) in ('M', 'MONTHLY', '1 - MONTHLY') then '1'
    when cast(payfreq as int) in (2, 26) or upper(trim(payfreq)) in ('B', 'BI-WEEKLY', 'BIWEEKLY', '2 - BI-WEEKLY') then '2'
    when cast(payfreq as int) in (3, 24) or upper(trim(payfreq)) in ('SEMI-MONTHLY', '3 - SEMI-MONTHLY', 'TWICE MONTHLY') then '3'
    when cast(payfreq as int) in (4, 52) or upper(trim(payfreq)) in ('W', 'WEEKLY', '4 - WEEKLY') then '4'
    when cast(payfreq as int) = 5 then '5'
    when cast(payfreq as int) = 6 then '6'
    else 'Unknown'
end as payfreq_cde,

/*employment type*/
CASE
  WHEN pdtcat IN (2, 3) THEN 'Not Available'
  WHEN CAST(borr.applicant_source_income_tcd AS INT) = 1 THEN 'Salaried'
  WHEN CAST(borr.applicant_source_income_tcd AS INT) = 3 THEN 'Self-employed'
  ELSE 'Other' /*2 = Seasonal, 5 = Pensioner, 7 = Alimony*/
END AS employ_grp,

iif.LNACCTNO as IIF_LNACCTNO,
iif.APLCAPRO as IIF_APLCAPRO,
--sum(iif.dwghtvar*iif.balance) as w_outbalance,
--sum(iif.lwghtvar*iif.totunit) as w_units,
--sum(iif.lwghtvar) as w_loans,
--iif.new_ltv2,
--iif.new_ltvhop2,

Case
when iif.new_ltvhop2 <= 50 THEN 'A <=50%        '
when 50 < iif.new_ltvhop2  and iif.new_ltvhop2 <= 55  THEN 'B >50% to <=55%' 
when 55 < iif.new_ltvhop2 and iif.new_ltvhop2<= 60  THEN 'C >55% to <=60%'
when 60 < iif.new_ltvhop2 and iif.new_ltvhop2<= 65  THEN 'D >60% to <=65%'
when 65 < iif.new_ltvhop2 and iif.new_ltvhop2<= 70  THEN 'E >65% to <=70%'
when 70 < iif.new_ltvhop2 and iif.new_ltvhop2<= 75  THEN 'F >70% to <=75%'
when 75 < iif.new_ltvhop2 and iif.new_ltvhop2<= 80  THEN 'G >75% to <=80%'
when 80 < iif.new_ltvhop2 and iif.new_ltvhop2<= 85  THEN 'H >80% to <=85%'
when 85 < iif.new_ltvhop2 and iif.new_ltvhop2<= 90  THEN 'I >85% to <=90%'
when 90 < iif.new_ltvhop2 and iif.new_ltvhop2<= 95  THEN 'J >90% to <=95%'
when 95 < iif.new_ltvhop2 and iif.new_ltvhop2<= 100 THEN 'K >95% to <=100%'
when iif.new_ltvhop2 > 100      THEN 'L >100%         '        
end as LTV_New_Grp,
/*Original LTV Combined*/
case 
	when preups = 1 then PRE_LNRTIOPC
	when port.lnrtiopc > 0 then port.lnrtiopc
	when crm.cmhc_loantovalue > 0 then crm.cmhc_loantovalue
	else las.loan_to_value_rtio
end as loan_to_value_ratio_comb,

/*variable ORIGINATION_LTV: loan to value at origination*/
case
	when loan_to_value_ratio_comb >  0 and loan_to_value_ratio_comb <= 50 then ' <= 50%'
	when loan_to_value_ratio_comb > 50 and loan_to_value_ratio_comb <= 65 then ' > 50 to <= 65%'
	when loan_to_value_ratio_comb > 65 and loan_to_value_ratio_comb <= 75 then ' > 65 to <= 75%'
	when loan_to_value_ratio_comb > 75 and loan_to_value_ratio_comb <= 80 then ' > 75 to <= 80%'
	when loan_to_value_ratio_comb > 80 and loan_to_value_ratio_comb <= 85 then ' > 80 to <= 85%'
	when loan_to_value_ratio_comb > 85 and loan_to_value_ratio_comb <= 90 then ' > 85 to <= 90%'
	when loan_to_value_ratio_comb > 90 and loan_to_value_ratio_comb <= 95 then ' > 90 to <= 95%'
	when loan_to_value_ratio_comb > 95 then ' > 95%'
	else 'Not Available'
end as ltv_origination_band,

/*Unit count integration. Removed CARL & ERIC values after analysis. They matched data warehouse.*/
case 
	when preups = 1 then PRE_TOTUTCT
	/*Portfolio*/
	when iif.pdtcat = 2 and port.tuntcnt > 0 then port.tuntcnt 
	/*Homeowner & multi*/
	when lad.unit_total_cnt > 0 then lad.unit_total_cnt
	else 1
end as unit_total_cnt_comb,

/*variable TDSBAND*/
case
	when pdtcat in (2,3) or debt_service_total_rtio is null then 'Not Available'
	when debt_service_total_rtio <= 30 then ' <= 30%'
	when debt_service_total_rtio > 30 and debt_service_total_rtio <= 35 then ' > 30 to <= 35%'
	when debt_service_total_rtio > 35 and debt_service_total_rtio <= 40 then ' > 35 to <= 40%'
	when debt_service_total_rtio > 40 and debt_service_total_rtio <= 44 then ' > 40 to <= 44%'
	when debt_service_total_rtio > 44 then ' > 44%'
end as tds_band,
/* variable GDSBAND */
case
	when pdtcat in (2,3) or debt_service_gross_rtio is null then 'Not Available'
	when debt_service_gross_rtio <= 30 then ' <= 30%'
	when debt_service_gross_rtio > 30 and debt_service_gross_rtio <= 35 then ' > 30 to <= 35%'
	when debt_service_gross_rtio > 35 and debt_service_gross_rtio <= 39 then ' > 35 to <= 39%' 
	when debt_service_gross_rtio > 39 then ' > 39%' 
end as gds_band,

/*The amortization variables have the format yymm. Convert them to years only
CASE
  WHEN TRIM(lad.loan_amortization_prd) <> '' THEN
    CAST(SUBSTR(lad.loan_amortization_prd, 1, 2) AS INT) +
    CAST(SUBSTR(lad.loan_amortization_prd, 3, 2) AS INT) / 12
  WHEN pdtcat = 2 THEN
    CAST(SUBSTR(port.rmamtzpd, 1, 2) AS INT) +
    CAST(SUBSTR(port.rmamtzpd, 3, 2) AS INT) / 12
  WHEN pdtcat = 3 THEN
    eric_amort_years_at_qual + (eric_amort_months_at_qual / 12)
END AS amort_yrs,*/

iif.remam/12 as rem_amort_years,

case
	when rem_amort_years is null then 'Not Available'
	when rem_amort_years <= 15 then ' <= 15 yrs'
	when rem_amort_years > 15 and rem_amort_years <= 20 then ' > 15 to <= 20 yrs'
	when rem_amort_years > 20 and rem_amort_years <= 25 then ' > 20 to <= 25 yrs'
	when rem_amort_years > 25 and rem_amort_years <= 30 then ' > 25 to <= 30 yrs'
	when rem_amort_years > 30 and rem_amort_years <= 35 then ' > 30 to <= 35 yrs'
	when rem_amort_years > 35 then ' > 35 yrs'
end as amort_band,

case
	when iif.remaining_term is null then 'Not Available'
	when iif.remaining_term <= 1 then ' <= 1 yr'
	when iif.remaining_term > 1 and iif.remaining_term <= 3 then ' > 1 to <= 3 yrs'
	when iif.remaining_term > 3 and iif.remaining_term <= 5 then ' > 3 to <= 5 yrs'
	when iif.remaining_term > 5 then ' > 5 yrs' 
end as int_term_band,

case
	when iif.balance>iif.totinsat and iif.cominsat=iif.orgloan then iif.cominsat
	else iif.totinsat
end as loan_insured_amt_comb,

/*variable AVLOANAMT: insured loan amount*/
case
	when loan_insured_amt_comb > 1000000 then ' > $1M'
	when loan_insured_amt_comb >  850000 and loan_insured_amt_comb <= 1000000 then ' > $850K to <= $1M'
	when loan_insured_amt_comb >  600000 and loan_insured_amt_comb <=  850000 then ' > $600K to <= $850K'
	when loan_insured_amt_comb >  400000 and loan_insured_amt_comb <=  600000 then ' > $400K to <= $600K'
	when loan_insured_amt_comb >  300000 and loan_insured_amt_comb <=  400000 then ' > $300K to <= $400K'
	when loan_insured_amt_comb >  200000 and loan_insured_amt_comb <=  300000 then ' > $200K to <= $300K'
	when loan_insured_amt_comb <= 200000 then ' <= $200K' 
	else 'Not Available' 
end as loan_amount_band,

/*NBW uses proplend exclusively, does not incorporate selling price.*/
case
	when preups = 1 then preups.pre_proplend
	when las.property_selling_price_amt > 1000 then las.property_selling_price_amt /*have seen invalid amounts like $1.00 and $100.00*/
	when las.property_lending_value_amt > 0 then las.property_lending_value_amt
	/*when eric_lending_value > 0 then eric_lending_value*/
	when port.propvaat > 0 then port.propvaat
	/*There were 13 multi-unit loans in Q3 2022 that had no value populated in either DW or ERIC, so use an estimate.*/
	when loan_to_value_ratio_comb > 0 and las.loan_requested_amt > 0 then las.loan_requested_amt /(loan_to_value_ratio_comb/100)
end as prop_value_comb,

/*property value*/
case
	when prop_value_comb is null then 'Not Available'
	when prop_value_comb <= 200000 then ' <= $200K'
	when prop_value_comb > 200000 and  prop_value_comb <= 300000 then ' > $200K to <= $300K'
	when  prop_value_comb > 300000 and  prop_value_comb <= 400000 then ' > $300K to <= $400K'
	when  prop_value_comb > 400000 and  prop_value_comb <= 600000 then ' > $400K to <= $600K'
	when  prop_value_comb > 600000 and  prop_value_comb <= 850000 then ' > $600K to <= $850K'
	when  prop_value_comb > 850000 then ' > $850K'
end as prop_value_band,

/*LOANPURPOSE*/
CASE
    WHEN pdtcat = 2 THEN 'Not Available'
    WHEN COALESCE(loan_purpose_tcd, PRE_LNPURPCD) = '1' THEN 'Purchase'
    WHEN COALESCE(loan_purpose_tcd, PRE_LNPURPCD) IN ('2', '3', '5', 'E', 'A') THEN 'Refinance'
    WHEN COALESCE(loan_purpose_tcd, PRE_LNPURPCD) IN ('D', '6') THEN 'Portability'
    ELSE crm.cmhc_loanpurpose_en
END AS loan_purpose,

COALESCE(crm.cmhc_previouscmhcaccountnumbername,lad.previous_loan_account_nbr) as previous_account_number,

case when substr(trim(iif.intype),1,1) in ('1','F','f') then 'Fixed' else 'Other'
end as interest_type2,

/*Max credit score*/
CASE
  WHEN pdtcat = 1 THEN GREATEST(empiric1, gencrsc1, empiric2, gencrsc2)
  WHEN pdtcat = 2 THEN GREATEST(CAST(port.gencrscr AS INT), CAST(port.cbgenscr AS INT))
END AS max_credit_score,

/*credit score band*/
case
	when pdtcat = 3 or max_credit_score is null then 'Not Available'
	when max_credit_score < 600 then ' >= 0 to < 600'
	when max_credit_score < 650 then ' >= 600 to < 650' 
	when max_credit_score < 700 then ' >= 650 to < 700'
	when max_credit_score < 750 then ' >= 700 to < 750'
	else ' >= 750'
end as credit_score_band,

/*income band*/
case
	when pdtcat in (2,3) or family_annual_income_amt is null then 'Not Available' 
	when las.family_annual_income_amt <= 25000 then ' <= 25K'
	when las.family_annual_income_amt >  25000 and las.family_annual_income_amt <=  50000 then ' > 25K to <= 50K'
	when las.family_annual_income_amt >  50000 and las.family_annual_income_amt <=  75000 then ' > 50K to <= 75K'
	when las.family_annual_income_amt >  75000 and las.family_annual_income_amt <= 100000 then ' > 75K to <= 100K'
	when las.family_annual_income_amt > 100000 and las.family_annual_income_amt <= 150000 then ' > 100K to <= 150K'
	when las.family_annual_income_amt > 150000 then ' > 150K'
end as income_band,

lend.parent_level1_lender_nm_en as admin_lender_level1_lender_nm_en,
lend.parent_level1_lender_cde as admin_lender_level1_lender_cde,
lend.parent_level2_lender_nm_en as admin_lender_level2_lender_nm_en,
lend.parent_level2_lender_cde as admin_lender_level2_lender_cde,

lend2.parent_level1_lender_nm_en as reporter_lender_level1_lender_nm_en,
lend2.parent_level1_lender_cde as reporter_lender_level1_lender_cde,
lend2.parent_level2_lender_nm_en as reporter_lender_level2_lender_nm_en,
lend2.parent_level2_lender_cde as reporter_lender_level2_lender_cde,

pr.current_stats_canada_province_nm_en,

/*POTFOLIO DATA*/
port2.PSRVLNCD AS INSURED_LENDER_CODE_PI,
port2.PSRVLNEN AS INSURED_LENDER_NAME_PI,

port.LNACCTNO AS CMHC_ACCOUNT_NBR_PI,
port.MTGPFACT AS SUBPOOL_NBR,
port.APLCRECD AS PI_RECEIVED_DATE,
port.APLCAPRO AS PI_APPROVED_DATE,
port.LENDLNNO AS PI_LENDER_LOAN_NUMBER,
port.PROPVAAT AS PI_Original_Lend_VALUE,
port.BRWNAM AS BORROWER_NAME_PI,
port.CBWNAM AS CO_BORROWER_NAME_PI,
port.BORBTHDT as BORROWER_BD_PI,
port.CBWBTHDT as CO_BORROWER_BD_PI,
port.LEGALDES AS PROPERTY_LEGAL_DESCRIPTION_PI,
port.DWELTYEN AS DWELLING_TYPE_DESCRIPTION_PI,
port.IADDT AS INTEREST_ADJUSTMENT_DATE_PI,
port.LNTMXDT AS LOAN_TERM_EXPIRY_dt_PI,
port.ORLNAMT AS ORIGINAL_LOAN_AMOUNT_PI,
port.LENDLNAT AS Current_LOAN_AMOUNT_PI,
port.OLTVRTPC AS ORIGINAL_LOAN_RATIO_PERCENTAGE_PI,
port.LNITRT AS LOAN_INTEREST_RATE_PI,
port.LNTRMPD AS LOAN_TERM_PERIOD_PI,
port.RMAMTZPD AS REMAINING_AMORTIZATION_PERIOD_PI,
port.TBEDCNT AS TOTAL_BED_COUNT_PI,
port.TUNTCNT AS TOTAL_Unit_COUNT_PI,

case when port.TNUCDE= "01" then "Freehold"
when port.TNUCDE="02" then "Leasehold"
when port.TNUCDE="03" then "Condo"
when port.TNUCDE="04" then "On Reserve"
when port.TNUCDE="05" then "Co-op"
when port.TNUCDE="06" then "Chattel"
when port.TNUCDE="09" then "Sale Agreement"
when port.TNUCDE="10" then "Rent with option to buy"
Else "Other" end as TENURE_CODE_PI,

concat_ws(' ',port.PAAPTNO, port.PASTNO, port.PASTNM, port.PROPSTTY, port.PROPSTDI, port.PAMUNINM, port.PAPOSTCD) AS PI_PROPERTY_ADDRESS,

port.ACSUCD AS ACCOUNT_STATUS_CODE_PI,

Case when port.INTTYPCD='01' then "fixed"
when port.INTTYPCD='02' then "Adjustable"
when port.INTTYPCD='03' then "Buydown"
when port.INTTYPCD='04' then "VRM"
when port.INTTYPCD='05' then "capped VRM"
Else "Other" end as Interest_type_cd_PI,

port.PRODTRID as Product_Tracker_cd_PI,
port.FMGSOSAM as fstMort_OS_at_2d_PI,
port.FMGCOSAM as fstMort_Cur_Balance_PI,
port.FMGACTNO as fsrMort_acc_num_PI,
port.LRACCTNO as Emili_LR_accnt_PI
/*,

case 
	when a.arrpefdt is not null then 1 else 0 end as ever_in_arrears
*/

FROM ds_ins_iif.ins_iif_raw_weighted iif

left join ds_ins_risk.ins_risk_last emili --TRANSACTIONAL
on iif.LNACCTNO = emili.lnacctno

left join cmwdw_insurance.vwdw_property_account_dim pr
    on iif.LNACCTNO = pr.cmhc_loan_account_nbr

LEFT JOIN cmwdw_insurance.vwdw_lender_dim lend
on iif.admlendr = lend.lender_cde

LEFT JOIN cmwdw_insurance.vwdw_lender_dim lend2
on iif.reporter = lend2.lender_cde

left join ds_ins_underwriting.ins_ups_mort port --PORTFOLIO
on iif.lnacctno=port.lnacctno

left join ds_ins_underwriting.ins_ups_prtf port2
on port.MTGPFACT=port2.mtgpfact

left join crm_mu.cmhc_muapplication crm --MULTI
on iif.LNACCTNO = crm.cmhc_name
left join crm_mu.cmhc_muprojectanalysis proj
on iif.LNACCTNO=proj.cmhc_name
Left join ds_ins_carl.ins_carl_loan carl
on iif.lnacctno = carl.lnacctno
left join crm_crosslob.cmhc_location loc 
on crm.cmhc_locationid = loc.cmhc_locationid 

left join cmwdw_insurance.vwdw_loan_account_dim lad
on iif.lnacctno = lad.cmhc_loan_account_nbr

Left join cmwdw_insurance.vwdw_loan_account_sum las 
ON (lad.loan_account_surrkey = las.loan_account_surrkey) 

left join cmwdw_insurance.vwdw_borrower_account_basic_dim borr
on las.first_borrower_account_surrkey = borr.borrower_account_surrkey

left join sb_asi.preups_dataset_clean preups
on iif.LNACCTNO = preups.LNACCTNO

where iif.report >= 20090101
and iif.report<>'20211231X'
and iif.iif_pop=1
and iif.pif=1



"""
)


In [0]:
df_01=spark.sql("""select

distinct crm.cmhc_previouscmhcaccountnumbername

FROM crm_mu.cmhc_muapplication crm

"""
)



In [0]:
df1.createOrReplaceTempView("df1")
df_01.createOrReplaceTempView("df_01")

In [0]:
df1_1=spark.sql("""Select 
df1.*,
case 
    when df1.IsLatestReportedDate=1 and df1.IIF_LNACCTNO =df_01.cmhc_previouscmhcaccountnumbername then 0 else 1
    end as Dropped_off_loan_NoRefi

From df1 as df1
left join df_01 as df_01
on df1.IIF_LNACCTNO=df_01.cmhc_previouscmhcaccountnumbername

"""
)

In [0]:
#testing
#filtered_df=df1.filter(col('prov_cde_comb')==2)
#filtered_df.display(100)

In [0]:
#df1.display()

In [0]:
new_location = "abfss://sandbox@eppd1stcdp01.dfs.core.windows.net/data/sandbox/SB_RMO_HO_Reporting/SB_RMO_HO_Reporting/iif_dashboard_new"
df1_1.write.option("path", new_location).mode("overwrite").format("orc").saveAsTable("sb_rmo_ho_reporting.iif_dashboard")



df1.write.mode("overwrite").format("orc").saveAsTable("sb_rmo_ho_reporting.iif_dashboard")
#print("Final Base Table with Channel, Source, BNS Contact row count: ", df_fiscal2.count())
print("Final Base Table with Channel, Source, BNS Contact creation completed at: ", str(datetime.now()))

In [0]:
#Arrears table creation

df2=spark.sql("""Select 
distinct a.lnacctno,
a.arrpefdt,
to_date (a.arrpefdt, 'yyyyMMdd') AS in_arrears_date,
case
  when cast(substr(a.arrpefdt,1,6) as int) - cast(substr(a.dtlstins,1,6) as int) >= 3 then 1
    else 0
end as arr_90day_ind,

case when r.TENURECD='04' then 1 else 0 end as on_reserve_ind,

premium_override_reason_txt,
case
    when 
        iif.pdtcat <> 2 and ( 
            trim(upper(premium_override_reason_txt)) in ('MINISTERIAL GUARA', 'MINIST. GUARANTEE', 'MINISTERIAL GTR', 'MUP GUARANTEE')
            or upper(premium_override_reason_txt) like '%RESERVE%' 
            or trim(upper(premium_override_reason_txt)) in ('FP', 'F/P')
            ) then 1
        else 0
end as prem_override_excl,

/*Loans backed by Indigenous and Northern Affairs Canada (INAC)
or Société d'habitation du Québec (SHQ)*/

case
    when a.cucode in ('0209', '0216', '0220', '0260', '0261', '0717', '0767', '0769', '0769', '0770') 
         or (a.cucode = '0025' and trim(upper(premium_override_reason_txt)) = 'SHQ') then 1
    else 0
end as cucode_excl,

/*Final filter that will be used to exclude arrears that are on reserve or backed by government guarantee*/

case 
	when on_reserve_ind = 1 
		or prem_override_excl = 1 
		or cucode_excl = 1 then 1
	else 0
end as arrears_to_excl

from ds_ins_servicing.ins_arrscurr a
left join ds_ins_risk.ins_risk_last r 
    on cast(a.lnacctno as int)=r.lnacctno
left join ds_ins_iif.ins_iif_raw_weighted iif
    on a.lnacctno=iif.lnacctno

left join cmwdw_insurance.vwdw_loan_account_dim lad
    on cast(a.lnacctno as int) = lad.cmhc_loan_account_nbr

left join cmwdw_insurance.vwdw_costable_unit_dim cu
    on a.cucode = cu.costable_unit_cde

where arrpefdt>=20091001
""")


In [0]:
df2.createOrReplaceTempView("df2")

In [0]:
#df2.display()

In [0]:
#remove exclusions
df3=spark.sql("""Select 
lnacctno,
arrpefdt,
in_arrears_date

from df2
where arrears_to_excl =0
""")

In [0]:
#df3.display()

In [0]:
df3.write.mode("overwrite").format("orc").saveAsTable("sb_rmo_ho_reporting.iif_dashboard_arrears")
#print("Final Base Table with Channel, Source, BNS Contact row count: ", df_fiscal2.count())
print("Final Base Table with Channel, Source, BNS Contact creation completed at: ", str(datetime.now()))

Final Base Table with Channel, Source, BNS Contact creation completed at:  2024-09-23 15:24:30.663550


In [0]:
#df3.count()